# Introduction
During my orchestra [LiTHe Blås](http://litheblas.org)'s 45 year aniversary I will be doing a customary "Skitsnack" between two songs. 
A "Skitsnack" is where an orchestra member keeps the audience preoccupied by talking about anything between heaven and earth until we start playing the next song.
I often find it hard to decide what I should talk about and usually end up telling really bad jokes or rambling about some algorithm I just read about. 

For this "Skitsnack" I will avoid coming up with my own material entierly by generating it with an n-grams model! 
The focus of this Notebook is simply to generate a short text which I think could entertain a group of 500 for about 30-60 seconds.
As such, any quantitative evaluation will only happen in the spur of the moment.

# Training data
I haven't decided what data I should use to train my model, and will probably have to try some different sources when the model is finished. The final text will definitely be in Swedish to accomodate the audience, but to get started I will use the Dansih author [H.C. Andersen's Fairy Tales](http://www.gutenberg.org/ebooks/32572) translated to English.

## Cleaning the data

In [1]:
import codecs

In [2]:
with codecs.open('hcandersen_fairy_tales.txt', 'r', encoding='utf-8') as f:
    text = f.read()

First let's remove all text which is not part of his stories, as I don't want to use this for training.

In [3]:
import re

In [4]:
for i in re.finditer(r"HANS ANDERSEN'S FAIRY TALES", text):
    print(i.start(0), i.end(0))
    # Hiding the output for future readability
    #print(text[i.start(0): i.end(0) + 200])
    #print("###########################")

627 654
4021 4048
4140 4167
375029 375056


In [5]:
text = text[4167:]

In [6]:
for i in re.finditer(r"NOTES", text):
    print(i.start(0), i.end(0))
    print(text[i.start(0): i.end(0) + 200])
    print("###########################")

367113 367118
NOTES


THE STORKS

          PAGE 29. On account of the ravages it makes among
          noxious animals, the stork is a privileged bird
          wherever it makes its home. In cities it is
     
###########################


In [7]:
text = text[:367113 ]

Let's remove all tabs and linebreaks.

In [8]:
text = re.sub(r'\r', '', text)
text = re.sub(r'\n', ' ', text)

## Tokenization
### Sentences
First let's parse the text as sentences using NLTK.

In [44]:
from nltk.tokenize import sent_tokenize

In [45]:
sentences = sent_tokenize(text)

In [46]:
print(sentences[0])
print(sentences[1])

     THE FLAX   THE flax was in full bloom; it had pretty little blue flowers, as delicate as the wings of a moth.
The sun shone on it and the showers watered it; and this was as good for the flax as it is for little children to be washed and then kissed by their mothers.


The first one is not good, it includes the story title. I will noth bother with this at the moment though, as this is not the text I will be using for my final results.

### Words

In [47]:
from nltk.tokenize import word_tokenize

In [48]:
sentences = list(map(word_tokenize, sentences))

In [49]:
print(sentences[1])

['The', 'sun', 'shone', 'on', 'it', 'and', 'the', 'showers', 'watered', 'it', ';', 'and', 'this', 'was', 'as', 'good', 'for', 'the', 'flax', 'as', 'it', 'is', 'for', 'little', 'children', 'to', 'be', 'washed', 'and', 'then', 'kissed', 'by', 'their', 'mothers', '.']


### Add beginning and end of sentence tags
All my sentences need a root and a stop token. I will insert a `<BOS>` tag before the first word of each sentence and replace the last punctuation of each sentence with `<EOS>`, I do this to distinguish between in sentence punctuation and the actual end of sentence. 

In [50]:
sentences = list(map(lambda x: ['<BOS>'] + x[:-1] + ['<EOS>'], sentences))

In [51]:
print(sentences[1])

['<BOS>', 'The', 'sun', 'shone', 'on', 'it', 'and', 'the', 'showers', 'watered', 'it', ';', 'and', 'this', 'was', 'as', 'good', 'for', 'the', 'flax', 'as', 'it', 'is', 'for', 'little', 'children', 'to', 'be', 'washed', 'and', 'then', 'kissed', 'by', 'their', 'mothers', '<EOS>']


# N-Gram model

## Constructing n-grams from the setnences
Let's try the nltk ngrams package

In [15]:
from nltk import ngrams

## Implementing a generative model
When generating my text I will not be considering context across sentences. Each `<EOS>` tag will be followed by a fresh `<BOS>`, interpreted as a unigram.

In [28]:
from functools import reduce
import numpy as np
import pandas as pd

In [157]:
class NGrams():
    def __init__(self, n):
        self.n = n
        
    def fit(self, sentences):
        # Count all ngrams
        self.grams = []
        for i in range(1, self.n+1):
            # Build a sorted list of all ngrams
            grams = list(map(lambda x: list(ngrams(x, i)), sentences))
            grams = np.array(reduce(lambda x, y: x + y, grams))
            grams = grams[np.lexsort(grams[:,::-1].T,),:]
            
            # Build an array marking the first unique occurence of a n-gram
            # Example unigrams, 
            # a = [('a'), ('a'), ('b'), ('c'), ('c')]
            # -> [True, False, True, True, False]
            first_occurence = np.append([True], np.array([np.any(grams[i-1] != grams[i]) for i in range(1, len(grams))]))
            
            # Assign each unique n-gram an index
            # Example unigrams, 
            # a = [('a'), ('a'), ('b'), ('c'), ('c')]
            # -> [0, 0, 1, 2, 2]
            ids = np.append([0], first_occurence[1:].cumsum())
            
            # Build a mapping from n-gram to count
            frequencies = list(zip(grams[first_occurence,:], np.bincount(ids)))
            self.grams.append(frequencies)
        
        # Find the frequency of all ngrams
        #self.frequencies = []
        #for i in range(1, self.n+1):
            
            
        

In [158]:
trigram = NGrams(3)

In [159]:
trigram.fit(sentences)

In [162]:
g = trigram.grams[0]

In [140]:
first_unique = np.append([True], np.array([np.any(g[i-1] != g[i]) for i in range(1, len(g))]))
ids = np.append([0], first_unique[1:].cumsum())

In [141]:
ids

array([    0,     1,     2, ..., 60861, 60862, 60863])

In [142]:
np.bincount(ids)

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [144]:
list(zip(g[first_unique,:], np.bincount(ids)))

[(array(['!', '--', 'the'],
        dtype='<U45'), 1), (array(['!', '_', "''"],
        dtype='<U45'), 1), (array(["'", "''", 'they'],
        dtype='<U45'), 1), (array(["'", ';', 'or'],
        dtype='<U45'), 1), (array(["'", 'I', 'asked'],
        dtype='<U45'), 1), (array(["'", 'I', 'thought'],
        dtype='<U45'), 1), (array(["'", 'Medici', '.'],
        dtype='<U45'), 1), (array(["'", '``', "'What"],
        dtype='<U45'), 1), (array(["'", 'altars', '<EOS>'],
        dtype='<U45'), 1), (array(["'", 'and', 'they'],
        dtype='<U45'), 1), (array(["'", 'apartments', ','],
        dtype='<U45'), 1), (array(["'", 'arms', ','],
        dtype='<U45'), 1), (array(["'", 'clear', 'eyes'],
        dtype='<U45'), 1), (array(["'", 'coats', 'was'],
        dtype='<U45'), 1), (array(["'", 'continued', 'the'],
        dtype='<U45'), 1), (array(["'", 'each', 'of'],
        dtype='<U45'), 1), (array(["'", 'hall', ','],
        dtype='<U45'), 1), (array(["'", 'hall', '<EOS>'],
        dtype='<

In [ ]:
np.append([0], np.any(np.diff(g, )))